# Segmenting and Clustering Neighborhoods in Toronto - Part 2| Week 3

###  Required to explore and cluster the Latitude & Longitude of neighborhoods in Toronto. 
#### (Using csv file)

Questions:
1. Follow part 1 for clean dataframe for neighborhoods. Saving part dataframe in csv and using it may also work fine.

2. Use the Geocoder package or the csv file to create the following dataframe:

<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/HZ3jNHNOEeiMwApe4i-fLg_f44f0f10ccfaf42fcbdba9813364e173_Screen-Shot-2018-06-18-at-7.18.16-PM.png?expiry=1604534400000&hmac=oQm5fZrnj1nx3JeaOe749x7gJrNSrpWFPe9qs5cuV8w" />


In [2]:
import requests
from bs4 import BeautifulSoup #package for scrapping
import pandas as pd

### Scrapping data from wikipedia page as wiki_data

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'html.parser') #outputs json data

In [4]:
table = wiki_data.find('tbody')  
rows = table.select('tr')             # getting precise location of tag for the required data from json
row = [r.get_text() for r in rows]    # geting required text rowwise

### Creating clean dataframe

#### Three columns: PostalCode, Borough, and Neighborhood

In [5]:
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df3.head()

,,Postal Code,,Borough,,Neighbourhood,
1,,M1A,,Not assigned,,Not assigned,
2,,M2A,,Not assigned,,Not assigned,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",


### Ignore cells with a borough that is Not assigned

In [6]:
df4 = df3[df3.Borough != 'Not assigned']
df4.head()

,,Postal Code,,Borough,,Neighbourhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",
6,,M6A,,North York,,"Lawrence Manor, Lawrence Heights",
7,,M7A,,Downtown Toronto,,"Queen's Park, Ontario Provincial Government",


### Combining neighborhoods into one row with the neighborhoods separated with a comma, for neighborhoods having the same postal code 

In [7]:
df5 = df4.groupby(['Postal Code', 'Borough'], sort = False).agg(','.join)
df5.reset_index(inplace = True)
df5.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Changing the value of the Neighborhood to be same like the value assigned to Borough

In [8]:
df6 = df5.replace("Not assigned", "Queen's Park")
df6.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Read csv file that has the geographical coordinates of each postal code.

In [9]:
url = "http://cocl.us/Geospatial_data"
df7 = pd.read_csv(url)
df7.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Creating new dataframe by Merging neighborhood data with lattitude & longitude data

In [11]:
df8 = pd.merge(df6, df7, on='Postal Code')
df8.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
